In [3]:
import os
import json
import pandas as pd
import numpy as np
import jinja2
pd.reset_option("display.max_rows")
pd.reset_option("display.max_columns")
pd.reset_option("display.width")
pd.set_option('display.max_colwidth', 100)  # default is 50

In [4]:
# Set the root directory containing the subdirectories and JSON files
root_dir = os.getcwd()

# Prepare a list to collect all the JSON data
data_list = []

# Walk through the directory tree
for subdir, _, files in os.walk(root_dir):
    for file in files:
        if file.endswith('.json'):
            json_path = os.path.join(subdir, file)
            with open(json_path, 'r') as f:
                try:
                    data = json.load(f)
                    data_list.append(data)
                except json.JSONDecodeError:
                    print(f"Failed to decode JSON: {json_path}")

# Convert to a DataFrame
df = pd.DataFrame(data_list)

df.head(6).sort_values(by=["run_id", "attempt"])


,model,prompt,exercise,output,token_count,input_tokens,output_tokens,cost,thinking_mode,compiles,...,display_name,language,metadata,is_release,git_version,$schema,title,description,type,definitions
0,o4-mini,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...","Section Exercise_2_7_3.\n\nNotation ""'d(' a , b )"" := (d_discrete_R a b) (format ""'d(' a , b ')...","Proof.\n\nTake a ∈ ℝ.\nWe need to show that ∀ b ∈ ℝ, (d(a, b) = 0) ⇒ (a = b).\n\nTake b ∈ ℝ.\nWe...",6793.0,4128.0,2665.0,0.016267,True,"File ""/tmp/E2_non_degenerate_2_crrdk2lf.v"", line 38, characters 21-24:\nError: Syntax error: ':'...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,gemini-2.5-flash-preview-04-17,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...","Section Exercise_2_7_3.\n\nNotation ""'d(' a , b )"" := (d_discrete_R a b) (format ""'d(' a , b ')...","```waterproof\nTake a ∈ ℝ.\nTake b ∈ ℝ.\nWe need to show that (d(a, b) = 0 ⇒ a = b).\nAssume tha...",4533.0,4405.0,128.0,0.000738,False,"File ""/tmp/E2_non_degenerate_2__uiezujn.v"", line 37, characters 0-1:\nError: Syntax Error: Lexer...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o3-mini,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...","Lemma not_closed : ¬ ([0,1) is _closed_).\nProof.","Proof.\nWe argue by contradiction.\nAssume that ([0,1) is _closed_).\nWe need to show that (1 ∈ ...",7041.0,4067.0,2974.0,0.017559,True,"File ""/tmp/tmpmr17n7q9.v"", line 33, characters 5-8:\nError: Syntax error: '.' expected after [co...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,o4-mini,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",Section Prop_6_2_2_i.\n\nVariable a : ℕ → ℝ.\nDefinition a_opp (n : ℕ) := - a(n).\n\nLemma exerc...,We show both directions.\n++ We need to show that (a _diverges to ∞_ ⇒ a_opp _diverges to -∞_).\...,7918.0,4119.0,3799.0,0.021247,True,"File ""/tmp/E6_8_2_ogxx0b44.v"", line 58, characters 44-45:\nError:\nIn environment\na : ℕ ⇨ ℝ\nH ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,o3-mini,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...",Section Prop_6_2_2_i.\n\nVariable a : ℕ → ℝ.\nDefinition a_opp (n : ℕ) := - a(n).\n\nLemma exerc...,Proof.\nWe show both directions.\n++ We need to show that (a _diverges to ∞_ ⇒ a_opp _diverges t...,6694.0,4119.0,2575.0,0.015861,True,"File ""/tmp/E6_8_2_l4y8kluj.v"", line 51, characters 4-19:\nError:\n`Take ...` can only be used to...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,o3-mini,"You are asked to write a proof in the syntax of Waterproof, a theorem proving educational softwa...","Lemma exercise_3_11_1 :\n ∃ M ∈ ℝ,\n ∀ x ∈ [0, 5], (x ≤ M).\nProof.","Lemma exercise_3_11_1 :\n ∃ M ∈ ℝ,\n ∀ x ∈ [0, 5], (x ≤ M).\nProof.\nChoose M := (5).\n* Ind...",5402.0,4086.0,1316.0,0.010285,True,"File ""/tmp/E3_11_1_7a9jvcqc.v"", line 37, characters 0-73:\nError: Nested proofs are discouraged ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
total_cost = df['cost'].sum()
total_cost

np.float64(1.6754445800000002)

In [5]:
# Total success rate per model
summary = df.groupby('model')['success'].mean().reset_index()
summary['success'] = (summary['success'] * 100).round(2)  # Convert to percentage
summary.rename(columns={'success': 'Success Rate (%)'}, inplace=True)

# Export to LaTeX
latex_table = summary.to_latex(index=False, column_format="ll", caption="Success rate per model", label="tab:success_rate")
print(latex_table)

\begin{table}
\caption{Success rate per model}
\label{tab:success_rate}
\begin{tabular}{ll}
\toprule
model & Success Rate (%) \\
\midrule
openai/o4-mini & 18.180000 \\
x-ai/grok-3-mini-beta & 16.670000 \\
\bottomrule
\end{tabular}
\end{table}



In [6]:
# Total success rate per exercise (check if this code is correct)
summary = df.groupby('exercise')['success'].mean().reset_index()
summary['success'] = (summary['success'] * 100).round(2)  # Convert to percentage
summary.rename(columns={'success': 'Success Rate (%)'}, inplace=True)

# Export to LaTeX
latex_table = summary.to_latex(index=False, column_format="ll", caption="Success rate per exercise", label="tab:success_rate")
print(latex_table)

\begin{table}
\caption{Success rate per exercise}
\label{tab:success_rate}
\begin{tabular}{ll}
\toprule
exercise & Success Rate (%) \\
\midrule
11_not_closed & 0.000000 \\
3_11_2 & 36.360000 \\
\bottomrule
\end{tabular}
\end{table}



In [7]:
# Grid of success rates per model and per exercise (requires multiple runs of all models)
grid = df.pivot_table(index='model', columns='exercise', values='success', aggfunc='mean') * 100
grid = grid.round(2)
grid

exercise,11_not_closed,3_11_2
model,,
openai/o4-mini,0.0,40.00
x-ai/grok-3-mini-beta,0.0,33.33


In [6]:
# Average output token counts per model
output_tokens_avg = df.groupby('model')['output_tokens'].mean().reset_index()
output_tokens_avg.rename(columns={'output_tokens': 'Avg Output Tokens'}, inplace=True)

# Average thinking token counts per model
thinking_tokens_avg = df.groupby('model')['thinking_tokens'].mean().reset_index()
thinking_tokens_avg.rename(columns={'thinking_tokens': 'Avg Thinking Tokens'}, inplace=True)

# Get the thinking_mode per model
thinking_mode_per_model = df[['model', 'thinking_mode']].drop_duplicates()

# Merge into the result
output_tokens_avg = output_tokens_avg.merge(thinking_mode_per_model, on='model')
output_tokens_avg = output_tokens_avg.merge(thinking_tokens_avg, on='model')

output_tokens_avg

,model,Avg Output Tokens,thinking_mode,Avg Thinking Tokens
0,anthropic/claude-3.7-sonnet,587.500000,False,NaN
1,anthropic/claude-3.7-sonnet:thinking,15999.000000,True,NaN
2,anthropic/claude-sonnet-4,441.500000,False,NaN
3,deepseek/deepseek-chat-v3-0324:free,273.333333,False,NaN
4,deepseek/deepseek-prover-v2:free,669.666667,False,NaN
5,deepseek/deepseek-r1:free,1688.333333,True,NaN
6,gemini-2.5-flash-preview-04-17,74.200000,False,NaN
7,gemini-2.5-pro-preview-05-06,131.000000,False,NaN
8,google/gemini-2.5-flash-preview-05-20,463.500000,False,NaN
9,google/gemini-2.5-flash-preview-05-20:thinking,4627.500000,True,NaN


In [7]:
# Average cost per model
cost_avg = df.groupby('model')['cost'].mean().reset_index()
cost_avg.rename(columns={'cost': 'Avg Cost ($)'}, inplace=True)
cost_avg

,model,Avg Cost ($)
0,anthropic/claude-3.7-sonnet,0.024696
1,anthropic/claude-3.7-sonnet:thinking,0.255953
2,anthropic/claude-sonnet-4,0.022506
3,deepseek/deepseek-chat-v3-0324:free,0.001377
4,deepseek/deepseek-prover-v2:free,0.003356
5,deepseek/deepseek-r1:free,0.006050
6,gemini-2.5-flash-preview-04-17,0.000679
7,gemini-2.5-pro-preview-05-06,0.006534
8,google/gemini-2.5-flash-preview-05-20,0.000901
9,google/gemini-2.5-flash-preview-05-20:thinking,0.003400


In [79]:
# Success rate per model compared across a few levels of tutorial verbosity
df['tutorial_len'] = df['tutorial'].apply(lambda x: len(x) if isinstance(x, str) else 0)

tutorial_success = df.groupby(['model', 'tutorial_len'])['success'].mean().reset_index()
tutorial_success['success'] = (tutorial_success['success'] * 100).round(2)
tutorial_success.rename(columns={'success': 'Success Rate (%)', 'tutorial_len': 'Tutorial Length'}, inplace=True)

tutorial_success

,model,Tutorial Length,Success Rate (%)
0,openai/o4-mini,8665,18.18
1,x-ai/grok-3-mini-beta,8665,16.67


In [80]:
# Success rate per model compared across a few levels of prompt verbosity
df['prompt_len'] = df['prompt'].apply(lambda x: len(x) if isinstance(x, str) else 0)

prompt_success = df.groupby(['model', 'prompt_len'])['success'].mean().reset_index()
prompt_success['success'] = (prompt_success['success'] * 100).round(2)
prompt_success.rename(columns={'success': 'Success Rate (%)', 'prompt_len': 'Prompt Length'}, inplace=True)

prompt_success

,model,Prompt Length,Success Rate (%)
0,openai/o4-mini,1294,18.18
1,x-ai/grok-3-mini-beta,1294,16.67


In [81]:
df_filtered = df[df['exercise'] == '6_8_1']
pd.set_option('display.max_colwidth', None)
df_filtered['errors'].to_frame()

,errors


In [56]:
df_filtered['line_with_error'].to_frame()

,line_with_error


In [82]:
import pandas as pd

# Group by run_id to get per-run summary
run_grouped = df.groupby('run_id').agg({
    'model': 'first',
    'success': list,
    'attempt': 'max',
    'max_attempts': 'first'
}).reset_index()

# Expand success list into per-k success map
max_k = df['max_attempts'].max()

for k in range(1, max_k + 1):
    def pass_at_k(row):
        successes = row['success']
        used_attempts = row['attempt']
        # Success in any of the first k attempts
        success_in_k = any(successes[:k])
        # Or the model didn't even use k attempts
        not_used_k = used_attempts < k
        return success_in_k or not_used_k

    run_grouped[f'pass@{k}'] = run_grouped.apply(pass_at_k, axis=1)

# Now compute per-model mean for each pass@k
passk_cols = [f'pass@{k}' for k in range(1, max_k + 1)]
passk_summary = run_grouped.groupby('model')[passk_cols].mean().reset_index()

# Convert to percentage
for col in passk_cols:
    passk_summary[col] = (passk_summary[col] * 100).round(2)

# Rename columns for display
passk_summary.rename(columns={col: f'Pass@{col[-1]} (%)' for col in passk_cols}, inplace=True)

passk_summary


,model,Pass@1 (%),Pass@2 (%),Pass@3 (%)
0,openai/o4-mini,25.0,50.0,50.0
1,x-ai/grok-3-mini-beta,50.0,50.0,50.0


In [ ]:
# TODO:
# thinking vs no thinking models (all)
# thinking vs no thinking on models that support both (fair)
# definition expanding comparison